# MedNeXt Inference Notebook
This notebook runs MedNeXt inference on multi‑modal brain MRI volumes, generates submission files, and provides optional visualization and verification tools.

**Sections**
1. Setup and configuration
2. Model loading
3. Dataset and preprocessing
4. Inference (sliding window + optional TTA)
5. Submission export
6. Optional visualization and verification

## 1. Setup
Clone the MedNeXt repository and install it in editable mode. Skip this section if the package is already available in your environment.

In [5]:
!git clone https://github.com/MIC-DKFZ/MedNeXt.git mednext
!pip install -e ./mednext

fatal: destination path 'mednext' already exists and is not an empty directory.
Obtaining file:///kaggle/working/mednext
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Attempting uninstall: mednextv1
    Found existing installation: mednextv1 1.7.0
    Uninstalling mednextv1-1.7.0:
      Successfully uninstalled mednextv1-1.7.0
  Running setup.py develop for mednextv1


## 2. Configuration and model loading
Set paths, model configuration, and load the checkpoint.

In [ ]:
import os
import sys
from typing import List
import numpy as np
import pandas as pd
import nibabel as nib
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

repo_path = os.path.abspath("mednext")
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

try:
    from nnunet_mednext import create_mednext_v1
except ImportError as e:
    raise ImportError(
        "MedNeXt import failed. Ensure the repository is cloned and installed."
    ) from e

MODEL_PATH = "/kaggle/input/mednext/pytorch/default/1/best_model.pt"
DATASET_PATH = "/kaggle/input/instant-odc-ai-hackathon/test"

class InferenceConfig:
    MODEL_SIZE = "B"
    KERNEL_SIZE = 3
    IN_CHANNELS = 4
    NUM_CLASSES = 4
    PATCH_SIZE = (128, 128, 128)
    USE_AMP = True
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_mednext_model(checkpoint_path, config):
    model = create_mednext_v1(
        num_input_channels=config.IN_CHANNELS,
        num_classes=config.NUM_CLASSES,
        model_id=config.MODEL_SIZE,
        kernel_size=config.KERNEL_SIZE,
        deep_supervision=False,
    )
    model.to(config.DEVICE)

    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

    checkpoint = torch.load(checkpoint_path, map_location=config.DEVICE, weights_only=False)
    if isinstance(checkpoint, dict) and "model_state_dict" in checkpoint:
        state_dict = checkpoint["model_state_dict"]
    else:
        state_dict = checkpoint
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model

inference_model = load_mednext_model(MODEL_PATH, InferenceConfig)

✅ MedNeXt library imported successfully!
Initializing model architecture...
Loading weights from /kaggle/input/mednext/pytorch/default/1/best_model.pt...
✅ Model loaded successfully (ignored auxiliary training heads)!
   (Best Validation Dice: 0.8268)


## 3. Dataset and preprocessing
Load multi‑modal volumes and apply robust intensity normalization.

In [ ]:
class InferenceDataset(Dataset):
    """
    Dataset for inference that loads full volumes without patch extraction.
    Auto-detects common modality naming patterns.
    """

    def __init__(self, data_dir: str, subject_ids: List[str] = None):
        self.data_dir = data_dir
        self.modality_patterns = [
            ["t1", "t1ce", "t2", "flair"],
            ["T1", "T1ce", "T2", "FLAIR"],
            ["T1", "T1CE", "T2", "FLAIR"],
            ["t1", "t1Gd", "t2", "flair"],
        ]
        if subject_ids is None and os.path.exists(data_dir):
            all_items = os.listdir(data_dir)
            self.subject_ids = sorted(
                [d for d in all_items if os.path.isdir(os.path.join(data_dir, d))]
            )
        else:
            self.subject_ids = subject_ids or []

        self.modalities = self._detect_modality_pattern()
        print(f"Subjects: {len(self.subject_ids)}")
        print(f"Modalities: {self.modalities}")

    def _detect_modality_pattern(self):
        if len(self.subject_ids) == 0:
            return ["t1", "t1ce", "t2", "flair"]

        subject_id = self.subject_ids[0]
        subject_dir = os.path.join(self.data_dir, subject_id)
        if not os.path.exists(subject_dir):
            return ["t1", "t1ce", "t2", "flair"]

        files = os.listdir(subject_dir)
        for pattern in self.modality_patterns:
            found_all = True
            for mod in pattern:
                possible_names = [
                    f"{subject_id}_{mod}.nii.gz",
                    f"{subject_id}-{mod}.nii.gz",
                    f"{subject_id}_{mod}.nii",
                    f"{mod}.nii.gz",
                    f"{mod}.nii",
                ]
                if not any(name in files for name in possible_names):
                    found_all = False
                    break
            if found_all:
                return pattern

        detected = []
        modality_keywords = {
            "t1ce": ["t1ce", "t1gd", "t1Gd", "T1CE", "T1Gd", "T1ce"],
            "t1": ["t1", "T1"],
            "t2": ["t2", "T2"],
            "flair": ["flair", "FLAIR", "Flair"],
        }
        for mod_key, keywords in modality_keywords.items():
            for f in files:
                f_lower = f.lower()
                if any(kw.lower() in f_lower for kw in keywords):
                    for kw in keywords:
                        if kw in f:
                            detected.append(kw)
                            break
                    break
        if len(detected) == 4:
            return detected
        return ["t1", "t1ce", "t2", "flair"]

    def __len__(self):
        return len(self.subject_ids)

    def _find_modality_file(self, subject_dir: str, subject_id: str, modality: str) -> str:
        files = os.listdir(subject_dir)
        patterns = [
            f"{subject_id}_{modality}.nii.gz",
            f"{subject_id}-{modality}.nii.gz",
            f"{subject_id}_{modality}.nii",
            f"{modality}.nii.gz",
            f"{modality}.nii",
        ]
        for pattern in patterns:
            if pattern in files:
                return os.path.join(subject_dir, pattern)
        modality_lower = modality.lower()
        for f in files:
            if modality_lower in f.lower() and ("nii" in f.lower()):
                if modality_lower == "t1":
                    if "t1ce" not in f.lower() and "t1gd" not in f.lower():
                        return os.path.join(subject_dir, f)
                else:
                    return os.path.join(subject_dir, f)
        raise FileNotFoundError(
            f"Could not find {modality} in {subject_dir}. Files: {files}"
        )

    def _load_nifti(self, filepath: str) -> np.ndarray:
        img = nib.load(filepath)
        return img.get_fdata().astype(np.float32)

    def _normalize(self, data: np.ndarray) -> np.ndarray:
        mask = data > 0
        if mask.sum() == 0:
            return data
        pixels = data[mask]
        p_low, p_high = np.percentile(pixels, 0.5), np.percentile(pixels, 99.5)
        data = np.clip(data, p_low, p_high)
        pixels = data[mask]
        mean, std = pixels.mean(), pixels.std()
        data = (data - mean) / (std + 1e-8)
        data[~mask] = 0
        return data

    def __getitem__(self, idx: int):
        subject_id = self.subject_ids[idx]
        subject_dir = os.path.join(self.data_dir, subject_id)
        modality_data = []
        for mod in self.modalities:
            filepath = self._find_modality_file(subject_dir, subject_id, mod)
            data = self._load_nifti(filepath)
            data = self._normalize(data)
            modality_data.append(data)
        volume = np.stack(modality_data, axis=0)

        seg = None
        seg_patterns = [
            f"{subject_id}_seg.nii.gz",
            f"{subject_id}-seg.nii.gz",
            "seg.nii.gz",
            f"{subject_id}_mask.nii.gz",
        ]
        files = os.listdir(subject_dir)
        for pattern in seg_patterns:
            if pattern in files:
                seg_path = os.path.join(subject_dir, pattern)
                seg = self._load_nifti(seg_path)
                new_seg = np.zeros_like(seg)
                new_seg[seg == 0] = 0
                new_seg[seg == 1] = 1
                new_seg[seg == 2] = 2
                new_seg[seg == 4] = 3
                seg = new_seg
                break

        volume_tensor = torch.from_numpy(volume.copy()).float()
        return {
            "subject_id": subject_id,
            "volume": volume_tensor,
            "ground_truth": seg,
            "original_shape": volume.shape[1:],
        }

inference_dataset = InferenceDataset(DATASET_PATH)
print(f"Inference dataset size: {len(inference_dataset)}")

Found 334 subjects in dataset
Sample subjects: ['BraTS2021_01333', 'BraTS2021_01334', 'BraTS2021_01335']
Files in first subject folder: ['BraTS2021_01333_t2.nii', 'BraTS2021_01333_t1.nii', 'BraTS2021_01333_t1ce.nii', 'BraTS2021_01333_flair.nii']
Detected modalities: ['t1', 't1ce', 't2', 'flair']

Inference dataset created with 334 subjects


## 4. Inference utilities
Sliding-window inference with optional test-time augmentation (TTA).

In [ ]:
def setup_multi_gpu(model):
    """Wrap model with DataParallel for multi-GPU inference."""
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    return model

inference_model = setup_multi_gpu(inference_model)

def rle_encode_c_order(mask):
    """Encode a binary mask using C-order flattening."""
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

def sliding_window_inference_logits(model, volume, config, overlap=0.6):
    """Return softmax probabilities for TTA accumulation."""
    patch_size = config.PATCH_SIZE
    device = config.DEVICE
    pd, ph, pw = patch_size
    C, D, H, W = volume.shape
    stride_d = int(pd * (1 - overlap))
    stride_h = int(ph * (1 - overlap))
    stride_w = int(pw * (1 - overlap))
    pad_d = max(0, pd - D)
    pad_h = max(0, ph - H)
    pad_w = max(0, pw - W)
    if pad_d > 0 or pad_h > 0 or pad_w > 0:
        volume = F.pad(volume, (0, pad_w, 0, pad_h, 0, pad_d))
        D, H, W = volume.shape[1:]
    output = torch.zeros((config.NUM_CLASSES, D, H, W), device=device)
    count = torch.zeros((D, H, W), device=device)
    d_pos = list(range(0, max(1, D - pd + 1), stride_d))
    h_pos = list(range(0, max(1, H - ph + 1), stride_h))
    w_pos = list(range(0, max(1, W - pw + 1), stride_w))
    if D > pd and D - pd not in d_pos:
        d_pos.append(D - pd)
    if H > ph and H - ph not in h_pos:
        h_pos.append(H - ph)
    if W > pw and W - pw not in w_pos:
        w_pos.append(W - pw)
    all_patches = []
    all_coords = []
    for d in d_pos:
        for h in h_pos:
            for w in w_pos:
                patch = volume[:, d : d + pd, h : h + ph, w : w + pw]
                all_patches.append(patch)
                all_coords.append((d, h, w))
    batch_size = max(1, torch.cuda.device_count() * 2)
    with torch.no_grad():
        for i in range(0, len(all_patches), batch_size):
            batch_patches = all_patches[i : i + batch_size]
            batch_coords = all_coords[i : i + batch_size]
            batch_tensor = torch.stack(batch_patches, dim=0).to(device)
            with torch.amp.autocast("cuda", enabled=config.USE_AMP):
                preds = model(batch_tensor)
            if isinstance(preds, (list, tuple)):
                preds = preds[0]
            preds = F.softmax(preds, dim=1)
            for j, (d, h, w) in enumerate(batch_coords):
                output[:, d : d + pd, h : h + ph, w : w + pw] += preds[j]
                count[d : d + pd, h : h + ph, w : w + pw] += 1
    output /= count.unsqueeze(0).clamp(min=1)
    original_d = D - pad_d if pad_d > 0 else D
    original_h = H - pad_h if pad_h > 0 else H
    original_w = W - pad_w if pad_w > 0 else W
    return output[:, :original_d, :original_h, :original_w]

def predict_with_tta(model, volume, config, overlap=0.6, use_tta=True):
    """Run sliding-window inference with optional TTA."""
    model.eval()
    logits_sum = sliding_window_inference_logits(model, volume, config, overlap=overlap)
    num_predictions = 1
    if use_tta:
        flip_axes = [[0], [1], [2]]
        for axes in flip_axes:
            flip_dim = [a + 1 for a in axes]
            flipped_vol = torch.flip(volume, dims=flip_dim)
            flipped_logits = sliding_window_inference_logits(
                model, flipped_vol, config, overlap=overlap
            )
            unflipped_logits = torch.flip(flipped_logits, dims=flip_dim)
            logits_sum += unflipped_logits
            num_predictions += 1
    avg_logits = logits_sum / num_predictions
    final_pred = torch.argmax(avg_logits, dim=0).cpu().numpy()
    return final_pred

def sliding_window_inference(model, volume, config, overlap=0.5):
    """Wrapper without TTA."""
    return predict_with_tta(model, volume, config, overlap=overlap, use_tta=False)

Sliding window inference function defined!


## 5. Run inference and export submission
Generate run-length encoding (RLE) per class and save to CSV.

In [ ]:
USE_TTA = True
OVERLAP = 0.6
CLEANUP_EVERY = 3

label_map = {1: 1, 2: 2, 3: 4}
submission_rows = []

inference_model.eval()
for idx in tqdm(range(len(inference_dataset)), desc="Inference"):
    sample = inference_dataset[idx]
    subject_id = sample["subject_id"]
    volume = sample["volume"]
    prediction = predict_with_tta(
        inference_model,
        volume,
        InferenceConfig,
        overlap=OVERLAP,
        use_tta=USE_TTA,
    )
    for model_label, comp_label in label_map.items():
        binary_mask = (prediction == model_label).astype(np.uint8)
        rle_string = rle_encode_c_order(binary_mask) if binary_mask.sum() > 0 else ""
        submission_rows.append({"id": f"{subject_id}_{comp_label}", "rle": rle_string})
    if idx % CLEANUP_EVERY == 0:
        torch.cuda.empty_cache()

submission_df = pd.DataFrame(submission_rows)
submission_df.to_csv("submission.csv", index=False)
print(f"Saved submission.csv with {len(submission_df)} rows")
submission_df.head(9)

Starting inference on 334 samples...


  0%|          | 0/334 [00:00<?, ?it/s]/tmp/ipykernel_55/4204801297.py:63: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=config.USE_AMP):
100%|██████████| 334/334 [37:46<00:00,  6.79s/it]



✅ Submission generated with 1002 rows.
                  id                                                rle
0  BraTS2021_01333_1  3668819 1 3854669 1 3891714 1 3891866 4 389201...
1  BraTS2021_01333_2  2399957 5 2400111 5 2400117 3 2400266 13 24004...
2  BraTS2021_01333_4  3668202 2 3668355 4 3668510 4 3668665 3 370524...
3  BraTS2021_01334_1  2250059 2 2287104 1 2287259 1 2287414 1 228756...
4  BraTS2021_01334_2  2064365 3 2064521 3 2064676 3 2064831 4 206498...


## 6. Optional: interactive visualization
Use this section to inspect predictions alongside MRI slices and (optionally) ground truth.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from ipywidgets import interact, IntSlider

class NestedFolderDataset(Dataset):
    """
    Dataset for nested folder structures with one subfolder per modality.
    """

    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        self.subject_ids = []
        if os.path.exists(data_dir):
            all_items = os.listdir(data_dir)
            self.subject_ids = sorted(
                [d for d in all_items if os.path.isdir(os.path.join(data_dir, d))]
            )
        print(f"Nested dataset subjects: {len(self.subject_ids)}")
        if self.subject_ids:
            print(f"Sample subjects: {self.subject_ids[:3]}")

    def __len__(self):
        return len(self.subject_ids)

    def _find_nifti_in_folder(self, folder_path):
        if not os.path.isdir(folder_path):
            return None
        for f in os.listdir(folder_path):
            if f.endswith(".nii.gz"):
                return os.path.join(folder_path, f)
        for f in os.listdir(folder_path):
            if f.endswith(".nii"):
                return os.path.join(folder_path, f)
        return None

    def _find_modality_file(self, patient_path, subject_id, modality):
        items = os.listdir(patient_path)
        modality_lower = modality.lower()
        for item in items:
            item_path = os.path.join(patient_path, item)
            item_lower = item.lower()
            if os.path.isdir(item_path):
                if modality_lower == "t1":
                    if ("t1" in item_lower and "t1ce" not in item_lower and "t1gd" not in item_lower):
                        nifti_file = self._find_nifti_in_folder(item_path)
                        if nifti_file:
                            return nifti_file
                elif modality_lower in item_lower:
                    nifti_file = self._find_nifti_in_folder(item_path)
                    if nifti_file:
                        return nifti_file
        direct_patterns = [
            os.path.join(patient_path, f"{subject_id}_{modality}.nii.gz"),
            os.path.join(patient_path, f"{subject_id}_{modality}.nii"),
            os.path.join(patient_path, f"{modality}.nii.gz"),
            os.path.join(patient_path, f"{modality}.nii"),
        ]
        for pattern in direct_patterns:
            if os.path.exists(pattern):
                return pattern
        raise FileNotFoundError(f"Could not find {modality} for {subject_id}")

    def _load_nifti(self, filepath):
        img = nib.load(filepath)
        return img.get_fdata().astype(np.float32)

    def _normalize(self, data):
        mask = data > 0
        if mask.sum() == 0:
            return data
        pixels = data[mask]
        p_low, p_high = np.percentile(pixels, 0.5), np.percentile(pixels, 99.5)
        data = np.clip(data, p_low, p_high)
        pixels = data[mask]
        mean, std = pixels.mean(), pixels.std()
        data = (data - mean) / (std + 1e-8)
        data[~mask] = 0
        return data

    def __getitem__(self, idx):
        subject_id = self.subject_ids[idx]
        patient_path = os.path.join(self.data_dir, subject_id)
        modalities = ["t1", "t1ce", "t2", "flair"]
        modality_data = []
        for mod in modalities:
            try:
                filepath = self._find_modality_file(patient_path, subject_id, mod)
                data = self._load_nifti(filepath)
                data = self._normalize(data)
                modality_data.append(data)
            except Exception:
                if modality_data:
                    modality_data.append(np.zeros_like(modality_data[0]))
                else:
                    raise
        volume = np.stack(modality_data, axis=0)
        seg = None
        try:
            seg_file = self._find_modality_file(patient_path, subject_id, "seg")
            seg = self._load_nifti(seg_file)
            new_seg = np.zeros_like(seg)
            new_seg[seg == 1] = 1
            new_seg[seg == 2] = 2
            new_seg[seg == 4] = 3
            seg = new_seg
        except Exception:
            pass
        volume_tensor = torch.from_numpy(volume.copy()).float()
        return {
            "subject_id": subject_id,
            "volume": volume_tensor,
            "ground_truth": seg,
            "original_shape": volume.shape[1:],
        }

TRAIN_PATH = "/kaggle/input/instant-odc-ai-hackathon/Train"
train_dataset = NestedFolderDataset(TRAIN_PATH)

def visualize_3d_by_id(subject_id, model=None, dataset=None, use_train=True):
    """Interactive 3D visualization for a specific patient."""
    if model is None:
        model = inference_model
    if dataset is None:
        dataset = train_dataset if use_train else inference_dataset
    try:
        patient_idx = dataset.subject_ids.index(subject_id)
    except ValueError:
        print(f"Subject not found: {subject_id}")
        print(f"Available subjects (first 10): {dataset.subject_ids[:10]}")
        return
    sample = dataset[patient_idx]
    volume = sample["volume"]
    ground_truth = sample["ground_truth"]
    has_gt = ground_truth is not None
    model.eval()
    with torch.no_grad():
        prediction = sliding_window_inference(
            model, volume, InferenceConfig, overlap=0.5
        )
    volume_np = volume.numpy()
    D, H, W = prediction.shape
    cmap = mcolors.ListedColormap(["none", "red", "green", "yellow"])
    bounds = [0, 0.5, 1.5, 2.5, 3.5]
    norm = mcolors.BoundaryNorm(bounds, cmap.N)

    def show_slices(axial_slice, sagittal_slice, coronal_slice):
        n_rows = 3 if has_gt else 2
        fig, axes = plt.subplots(n_rows, 3, figsize=(18, 6 * n_rows))
        flair_ax = volume_np[3, :, :, axial_slice]
        flair_sag = volume_np[3, sagittal_slice, :, :]
        flair_cor = volume_np[3, :, coronal_slice, :]
        pred_ax = prediction[:, :, axial_slice]
        pred_sag = prediction[sagittal_slice, :, :]
        pred_cor = prediction[:, coronal_slice, :]
        axes[0, 0].imshow(flair_ax, cmap="gray")
        axes[0, 0].set_title(f"Axial (Z={axial_slice})")
        axes[0, 0].axis("off")
        axes[0, 1].imshow(flair_sag.T, cmap="gray", origin="lower")
        axes[0, 1].set_title(f"Sagittal (X={sagittal_slice})")
        axes[0, 1].axis("off")
        axes[0, 2].imshow(flair_cor.T, cmap="gray", origin="lower")
        axes[0, 2].set_title(f"Coronal (Y={coronal_slice})")
        axes[0, 2].axis("off")
        axes[1, 0].imshow(flair_ax, cmap="gray", alpha=0.7)
        axes[1, 0].imshow(pred_ax, cmap=cmap, norm=norm, alpha=0.6)
        axes[1, 0].set_title("Axial - Prediction")
        axes[1, 0].axis("off")
        axes[1, 1].imshow(flair_sag.T, cmap="gray", origin="lower", alpha=0.7)
        axes[1, 1].imshow(pred_sag.T, cmap=cmap, norm=norm, origin="lower", alpha=0.6)
        axes[1, 1].set_title("Sagittal - Prediction")
        axes[1, 1].axis("off")
        axes[1, 2].imshow(flair_cor.T, cmap="gray", origin="lower", alpha=0.7)
        axes[1, 2].imshow(pred_cor.T, cmap=cmap, norm=norm, alpha=0.6)
        axes[1, 2].set_title("Coronal - Prediction")
        axes[1, 2].axis("off")
        if has_gt:
            gt_ax = ground_truth[:, :, axial_slice].astype(int)
            gt_sag = ground_truth[sagittal_slice, :, :].astype(int)
            gt_cor = ground_truth[:, coronal_slice, :].astype(int)
            axes[2, 0].imshow(flair_ax, cmap="gray", alpha=0.7)
            axes[2, 0].imshow(gt_ax, cmap=cmap, norm=norm, alpha=0.6)
            axes[2, 0].set_title("Axial - Ground Truth")
            axes[2, 0].axis("off")
            axes[2, 1].imshow(flair_sag.T, cmap="gray", origin="lower", alpha=0.7)
            axes[2, 1].imshow(gt_sag.T, cmap=cmap, norm=norm, origin="lower", alpha=0.6)
            axes[2, 1].set_title("Sagittal - Ground Truth")
            axes[2, 1].axis("off")
            axes[2, 2].imshow(flair_cor.T, cmap="gray", origin="lower", alpha=0.7)
            axes[2, 2].imshow(gt_cor.T, cmap=cmap, norm=norm, alpha=0.6)
            axes[2, 2].set_title("Coronal - Ground Truth")
            axes[2, 2].axis("off")
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor="red", label="NCR/NET (1)"),
            Patch(facecolor="green", label="Edema (2)"),
            Patch(facecolor="yellow", label="Enhancing (4)"),
        ]
        fig.legend(handles=legend_elements, loc="lower center", ncol=3, fontsize=11)
        plt.suptitle(f"Patient: {subject_id}", fontsize=14, fontweight="bold")
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.05)
        plt.show()

    tumor_per_axial = (prediction > 0).sum(axis=(0, 1))
    tumor_per_sagittal = (prediction > 0).sum(axis=(1, 2))
    tumor_per_coronal = (prediction > 0).sum(axis=(0, 2))
    default_ax = int(np.argmax(tumor_per_axial)) if tumor_per_axial.max() > 0 else W // 2
    default_sag = int(np.argmax(tumor_per_sagittal)) if tumor_per_sagittal.max() > 0 else D // 2
    default_cor = int(np.argmax(tumor_per_coronal)) if tumor_per_coronal.max() > 0 else H // 2
    interact(
        show_slices,
        axial_slice=IntSlider(min=0, max=W - 1, step=1, value=default_ax, description="Axial (Z):"),
        sagittal_slice=IntSlider(min=0, max=D - 1, step=1, value=default_sag, description="Sagittal (X):"),
        coronal_slice=IntSlider(min=0, max=H - 1, step=1, value=default_cor, description="Coronal (Y):"),
    )
    return prediction

def list_train_subjects(n=20):
    print(f"Training subjects: {len(train_dataset.subject_ids)}")
    for i, sid in enumerate(train_dataset.subject_ids[:n]):
        print(f"{i}: {sid}")

list_train_subjects(10)
# visualize_3d_by_id("BraTS2021_00005", use_train=True)
# visualize_3d_by_id("BraTS2021_01333", use_train=False)

Found 917 subjects in nested dataset
Sample subjects: ['BraTS2021_00000', 'BraTS2021_00002', 'BraTS2021_00003']
Subfolders in first patient: ['BraTS2021_00000_seg.nii', 'BraTS2021_00000_flair.nii', 'BraTS2021_00000_t1ce.nii', 'BraTS2021_00000_t2.nii', 'BraTS2021_00000_t1.nii']
Files in 'BraTS2021_00000_seg.nii': ['00000057_final_seg.nii']
✅ Training dataset loaded with 917 subjects (with ground truth)
📋 Total training subjects: 917
First 10 subjects:
   0: BraTS2021_00000
   1: BraTS2021_00002
   2: BraTS2021_00003
   3: BraTS2021_00005
   4: BraTS2021_00006
   5: BraTS2021_00008
   6: BraTS2021_00009
   7: BraTS2021_00011
   8: BraTS2021_00012
   9: BraTS2021_00014
🔍 Loading patient: BraTS2021_01333
   Ground truth available: ❌ No
🧠 Running inference...


/tmp/ipykernel_55/4204801297.py:63: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=config.USE_AMP):


✅ Inference complete! Volume shape: (240, 240, 155)
   Predicted classes: [0 1 2 3]


interactive(children=(IntSlider(value=123, description='Axial (Z):', max=154), IntSlider(value=102, descriptio…


📈 Tumor Statistics (Prediction):
   NCR/NET: 43 voxels
   Edema: 3,901 voxels
   Enhancing: 1,099 voxels


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

## 7. Optional: RLE verification
Compare model predictions with decoded RLE from the submission file.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from ipywidgets import interact, IntSlider

SUBMISSION_PATH = "/kaggle/working/submission.csv"
submission_df = pd.read_csv(SUBMISSION_PATH)
print(f"Loaded submission.csv with {len(submission_df)} rows")
submission_df.head(10)

def rle_decode(rle_string, shape):
    """Decode RLE string back to a binary mask (C-order)."""
    if pd.isna(rle_string) or rle_string == "" or rle_string.strip() == "":
        return np.zeros(shape, dtype=np.uint8)
    s = rle_string.split()
    starts = np.array(s[0::2], dtype=int)
    lengths = np.array(s[1::2], dtype=int)
    total_size = np.prod(shape)
    mask_1d = np.zeros(total_size, dtype=np.uint8)
    for start, length in zip(starts, lengths):
        mask_1d[start : start + length] = 1
    mask = mask_1d.reshape(shape)
    return mask

def verify_encoding_by_id(subject_id, model=None, dataset=None, slice_idx=None):
    """Compare model prediction with decoded RLE for a subject."""
    if model is None:
        model = inference_model
    if dataset is None:
        dataset = inference_dataset
    try:
        patient_idx = dataset.subject_ids.index(subject_id)
    except ValueError:
        print(f"Subject not found: {subject_id}")
        print(f"Available subjects (first 10): {dataset.subject_ids[:10]}")
        return
    sample = dataset[patient_idx]
    volume = sample["volume"]
    with torch.no_grad():
        prediction = sliding_window_inference(
            model, volume, InferenceConfig, overlap=0.5
        )
    label_map = {1: 1, 2: 2, 4: 3}
    decoded_masks = {}
    for comp_label, model_label in label_map.items():
        row_id = f"{subject_id}_{comp_label}"
        row = submission_df[submission_df["id"] == row_id]
        if len(row) == 0:
            decoded_masks[model_label] = np.zeros(prediction.shape, dtype=np.uint8)
        else:
            rle_string = row["rle"].values[0]
            decoded_masks[model_label] = rle_decode(rle_string, prediction.shape)
    decoded_combined = np.zeros(prediction.shape, dtype=np.uint8)
    for model_label, mask in decoded_masks.items():
        decoded_combined[mask == 1] = model_label
    exact_match = np.array_equal(prediction, decoded_combined)
    print(f"Exact match: {exact_match}")
    if not exact_match:
        diff = (prediction != decoded_combined)
        print(f"Differing voxels: {diff.sum()} / {prediction.size}")
    if slice_idx is None:
        tumor_per_slice = (prediction > 0).sum(axis=(0, 1))
        slice_idx = int(np.argmax(tumor_per_slice)) if tumor_per_slice.max() > 0 else prediction.shape[2] // 2
    volume_np = volume.numpy()
    flair = volume_np[3]
    cmap = mcolors.ListedColormap(["none", "red", "green", "yellow"])
    bounds = [0, 0.5, 1.5, 2.5, 3.5]
    norm = mcolors.BoundaryNorm(bounds, cmap.N)

    def show_comparison(slice_idx):
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        flair_slice = flair[:, :, slice_idx]
        pred_slice = prediction[:, :, slice_idx]
        decoded_slice = decoded_combined[:, :, slice_idx]
        axes[0, 0].imshow(flair_slice, cmap="gray")
        axes[0, 0].set_title(f"FLAIR (slice {slice_idx})")
        axes[0, 0].axis("off")
        axes[0, 1].imshow(flair_slice, cmap="gray", alpha=0.7)
        axes[0, 1].imshow(pred_slice, cmap=cmap, norm=norm, alpha=0.6)
        axes[0, 1].set_title("Model prediction")
        axes[0, 1].axis("off")
        axes[0, 2].imshow(flair_slice, cmap="gray", alpha=0.7)
        axes[0, 2].imshow(decoded_slice, cmap=cmap, norm=norm, alpha=0.6)
        axes[0, 2].set_title("Decoded from CSV")
        axes[0, 2].axis("off")
        for i, (model_label, name) in enumerate([(1, "NCR/NET"), (2, "Edema"), (3, "Enhancing")]):
            pred_class = (prediction[:, :, slice_idx] == model_label).astype(int)
            decoded_class = (decoded_combined[:, :, slice_idx] == model_label).astype(int)
            diff_img = np.zeros((*pred_class.shape, 3))
            diff_img[pred_class & decoded_class] = [0, 1, 0]
            diff_img[pred_class & ~decoded_class] = [1, 0, 0]
            diff_img[~pred_class & decoded_class] = [0, 0, 1]
            axes[1, i].imshow(flair_slice, cmap="gray", alpha=0.5)
            axes[1, i].imshow(diff_img, alpha=0.7)
            match_pct = 100 * (pred_class == decoded_class).sum() / pred_class.size
            axes[1, i].set_title(f"{name}\n{match_pct:.2f}% match")
            axes[1, i].axis("off")
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor="red", label="NCR/NET (1)"),
            Patch(facecolor="green", label="Edema (2)"),
            Patch(facecolor="yellow", label="Enhancing (4)"),
        ]
        fig.legend(handles=legend_elements, loc="lower center", ncol=3, fontsize=11)
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.08)
        plt.show()

    interact(
        show_comparison,
        slice_idx=IntSlider(min=0, max=prediction.shape[2] - 1, step=1, value=slice_idx, description="Slice:"),
    )
    return prediction, decoded_combined, exact_match

def list_test_subjects(n=20):
    print(f"Test subjects: {len(inference_dataset.subject_ids)}")
    for i, sid in enumerate(inference_dataset.subject_ids[:n]):
        print(f"{i}: {sid}")

list_test_subjects(5)
# verify_encoding_by_id(inference_dataset.subject_ids[2])

✅ Loaded submission.csv with 1002 rows
                  id                                                rle
0  BraTS2021_01333_1  3668819 1 3854669 1 3891714 1 3891866 4 389201...
1  BraTS2021_01333_2  2399957 5 2400111 5 2400117 3 2400266 13 24004...
2  BraTS2021_01333_4  3668202 2 3668355 4 3668510 4 3668665 3 370524...
3  BraTS2021_01334_1  2250059 2 2287104 1 2287259 1 2287414 1 228756...
4  BraTS2021_01334_2  2064365 3 2064521 3 2064676 3 2064831 4 206498...
5  BraTS2021_01334_4  2174885 1 2175508 2 2175663 3 2175818 3 221162...
6  BraTS2021_01335_1  5788252 2 5788406 3 5788561 3 5788717 2 578964...
7  BraTS2021_01335_2  4818423 1 4818577 2 4818731 3 4818885 4 481904...
8  BraTS2021_01335_4  5305733 2 5342933 1 5379978 1 5380133 1 541717...
9  BraTS2021_01336_1  5113240 1 5113394 2 5113538 1 5113548 3 511370...

RLE ENCODING VERIFICATION
📋 Total test subjects: 334
First 5 subjects:
   0: BraTS2021_01333
   1: BraTS2021_01334
   2: BraTS2021_01335
   3: BraTS2021_01336
   4: Bra

/tmp/ipykernel_55/4204801297.py:63: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=config.USE_AMP):


   Prediction shape: (240, 240, 155)
   BraTS2021_01334_1: Decoded 1,403 voxels
   BraTS2021_01334_2: Decoded 48,530 voxels
   BraTS2021_01334_4: Decoded 12,179 voxels

📊 COMPARISON:
   Class             Model Pred  Decoded CSV    Match
   --------------------------------------------------
   NCR/NET                1,403        1,403        ✅
   Edema                 48,530       48,530        ✅
   Enhancing             12,179       12,179        ✅

   Exact pixel-by-pixel match: ❌ NO
   Differing voxels: 8,938 / 8,928,000


interactive(children=(IntSlider(value=93, description='Slice:', max=154), Output()), _dom_classes=('widget-int…

(array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],